# RFO

Relative Requency of Occurance (RFO) for a WS is calculated as:

$RFO = \frac{\text{Gridbox counts for single WS}}{\text{Gridbox counts across all WS}}$

RFO tells us how much of a given WS makes up the total data. 

# RFO for Equal Area Map

As seen in `h1_wsmaps.ipynb`, the calculation above is done across all data for every gridbox. Plotting the result is fine and accurate -- however, we are also interested in the RFO on a global scale, not just gridbox by gridbox. To do this, it would make sense to add up the gridbox counts for each WS and divide by the total counts across all WS'. This is technically incorrect because the maps are in Equal-Area format. This would incorrectly give an area in the north pole equal weight to an area near the equator, giving an inaccurate RFO.

To handle this issue, global RFO, or $RFO_{weight}$, is calculated a different way. First, create the WS maps as done in `h1_wsmaps.ipynb` such that each gridbox, or cell of the resulting array/map, is an RFO value for that gridbox. Next, define the following equation: 

$$RFO_{weight} = \frac{\sum_{\theta=lat} \cos{\theta}*E(\theta)}{\sum_{\theta=lat} \cos{\theta}}$$

where $E(\theta)$ is the expected value, or average, gridbox RFO across an entire latitude given by angle $\theta$. 
Running every WS map through this equation will produce a single number representing the global RFO for a given WS.

Note that each gridbox is a square of dimension 1 degree by 1 degree (that can and will vary). As such, $\theta$ will be the middle of each gridbox. This means the top most gridboxes bordering the north pole will have $\theta = 89.5^{\circ}$, followed by $\theta = 88.5^{\circ}$, $\theta = 87.5^{\circ}$, etc. until $\theta = -89.5^{\circ}$.

The translation to code is below, along with an example of use. This function can be found in `api.py`.

In [8]:
import os
import gc
import math
import numpy as np

In [1]:
# Create function to compute Weighted RFO

def rfo(maps):
    """
    Compute total RFO of each WS for given data in 
    Equal-Area format.
    
    Attributes
    ----------
    maps: 2D numpy array
        WS maps with gridbox in terms of RFO. Shape of (12, 64800).
    """
    # Initialize emtpy list to hold RFO for each WS
    totallist = []
    
    # List of latitudinal angles (middle of gridbox)
    latbox = np.arange(.5, 90, 1.0)
    
    # Loop across each WS
    for i in range(12):
        total = 0
        weightsum = 0
        tmp_map = maps[i,:].reshape(360, 180)
        # Top half of map (North Pole to Equator)
        for j, lat in enumerate(latbox[::-1]):
            # Check if all gridboxes are emtpy, if so skip
            if np.all(tmp_map[:,j].mask):
                continue
            avg = np.mean(tmp_map[:,j])
            total += avg*math.cos(math.radians(lat))
            weightsum += math.cos(math.radians(lat))
        # Bottom half of map (Equator to South Pole)
        for j, lat in enumerate(latbox):
            if np.all(tmp_map[:,(j+90)].mask):
                continue
            avg = np.mean(tmp_map[:,(j+90)])
            total += avg*math.cos(math.radians(lat))
            weightsum += math.cos(math.radians(lat))
        totallist.append(total/(weightsum))
    return totallist

In [6]:
path = '/run/media/dtropf/Seagate Backup Plus Drive/validation/testing_03_04_2018/fortran_fnorm_new/H1_'
data = np.zeros((12,12,360*180))
years = [str(i) for i in range(1984, 2010)]

for year in years:
    h1data = np.load(path+year+'.npy')
    data[:,:,:] = data[:,:,:]+h1data[:,:,:]
    del h1data
    gc.collect()
    
total_data = np.sum(np.sum(data,axis=0),axis=0)
masked_total = np.ma.masked_where(total_data<=0, total_data)
masked_data = np.ma.masked_where(np.sum(data,axis=0)<=0, np.sum(data,axis=0))
normed_data = np.ma.zeros((12,360*180))
for i in range(12):
    normed_data[i,:] = masked_data[i,:]/total_data

In [10]:
# Compute RFO for each global WS
rfo_list = rfo(normed_data)

# Print list of RFOs
print(rfo_list)

# Confirm that sum = 1
print(sum(rfo_list))

[0.040695751977692325, 0.0752969732405213, 0.03669755822001327, 0.0575233103850647, 0.1074620579821253, 0.11319565830982359, 0.1996981188120116, 0.09333678071531365, 0.03522068253349227, 0.0669192809102082, 0.029291407024400702, 0.14842014483190277]
1.0037577249425695
